In [1]:
import os
import re
from pathlib import Path

os.chdir('../')

In [2]:
PT_VERBS = ['tratar', 'estabelecer', 'marcar', 'vir', 'colocar',\
           'fechar', 'dar', 'cair', 'encontrar', 'registrar',\
           'levar', 'receber', 'apresentar', 'passar', 'deixar',\
           'chegar', 'ficar', 'fazer', 'ter', 'ser']

In [3]:
def list_files(startpath):
    selected_files = []
    for root, dirs, files in os.walk(startpath):
        level = root
        for f in files:
            if 'dls/verb' in level and '.xml' in f and 'ontology' not in f:
                selected_files.append(level + '/' + f)
    return selected_files

In [4]:
selected_files = list_files(Path.cwd() / 'data/CSTNews 6.0')

In [5]:
verbs_file = {}
id_synstes = []
with open(Path.cwd() / 'data/wordnet/index.sense') as f:
    for ix, line in enumerate(f.read().split('\n')):
        if len(line) > 1:
            verbs_file[line.split()[0]] = line.split()[1]
            id_synstes.append(line.split()[1])
f.close()
id_synstes = list(set(id_synstes))

In [6]:
input_sentences = []
stem_sentences = []
output_sentences = []
output_selected_verb_sentences = []
selected_arrays = []
total_anns = 0
s = []
for name_file in selected_files:
    with open(name_file, 'r') as f:
        xml = f.read().lower()
        texts = re.findall(r'<texto(.*?)</texto>', xml, re.DOTALL)
        
        for text in texts:
            sentences = re.findall(r'<p>(.*?)</p>', text, re.DOTALL)
            for sentence in sentences:
                tokens = re.findall(r'<token>(.*?)</token>', text, re.DOTALL)
                temp_in = []
                temp_out = []
                temp_stem = []
                temp_sel = []
                temp = []
                for token in tokens:
                    word_value = re.findall(r'<valor>(.*?)</valor>', token, re.DOTALL)
                    sense_value = re.findall(r'<synset selecionado="true">(.*?)</synset>', token, re.DOTALL)
                    stem_value = re.findall(r'<lema>(.*?)</lema>', token, re.DOTALL)
                    
                    
                    temp_stem.append(stem_value[0])
                    if len(sense_value) > 0:
                        temp.append(word_value[0] + '<' + (8 - len(sense_value[0])) * '0' + sense_value[0] + '>')
                        if stem_value[0] in PT_VERBS:
                            temp_in.append(stem_value[0] + '_tag')
                            temp_sel.append((8 - len(sense_value[0])) * '0' + sense_value[0])
                            s.append((8 - len(sense_value[0])) * '0' + sense_value[0])
                        else:
                            temp_in.append(word_value[0])
                            temp_sel.append(word_value[0])
                        temp_out.append((8 - len(sense_value[0])) * '0' + sense_value[0])
                    else:
                        temp.append(word_value[0])
                        temp_in.append(word_value[0])
                        temp_sel.append(word_value[0])
                        temp_out.append(word_value[0])
                        
                selected_arrays.append(' '.join(temp))
                input_sentences.append(' '.join(temp_in))
                output_sentences.append(' '.join(temp_out))
                stem_sentences.append(' '.join(temp_stem))
                output_selected_verb_sentences.append(' '.join(temp_sel))

In [7]:
sentences = list(zip(input_sentences, output_sentences, output_selected_verb_sentences))
sentences = list(set(sentences))

In [11]:
f_in_all = open(Path.cwd() / 'data/disambiguation/test/all_verbs_lemma_in.txt', 'w')
f_out_all = open(Path.cwd() / 'data/disambiguation/test/all_verbs_lemma_out.txt', 'w') 
f_in_sel = open(Path.cwd() / 'data/disambiguation/test/verbs_selected_lemma_in.txt', 'w')
f_out_sel = open(Path.cwd() / 'data/disambiguation/test/verbs_selected_lemma_out.txt', 'w')
r = 0
for input_s, output_s_all, output_s_sel in sentences:
    #print(input_s)
    input_s = ' '.join(input_s.split()[1:])
    output_s_all = ' '.join(output_s_all.split()[1:])
    output_s_sel = ' '.join(output_s_sel.split()[1:])
    
    for i_s, o_a, o_s in zip(input_s.split('.'), output_s_all.split('.'), output_s_sel.split('.')):
        if len(i_s) > 4:
            f_in_all.write(i_s + '\n')
            f_out_all.write(o_a + '\n')

            f_in_sel.write(i_s + '\n')
            f_out_sel.write(o_s + '\n')
            if '_tag' in i_s:
                r += 1
    
f_in_all.close()
f_out_all.close()
f_in_sel.close()
f_out_sel.close()